In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
columns={'spacegroup' : 'sg',
                            'number_of_total_atoms' : 'Natoms',
                            'percent_atom_al' : 'x_Al',
                            'percent_atom_ga' : 'x_Ga',
                            'percent_atom_in' : 'x_In',
                            'lattice_vector_1_ang' : 'a',
                            'lattice_vector_2_ang' : 'b',
                            'lattice_vector_3_ang' : 'c',
                            'lattice_angle_alpha_degree' : 'alpha',
                            'lattice_angle_beta_degree' : 'beta',
                            'lattice_angle_gamma_degree' : 'gamma',
                            'formation_energy_ev_natom' : 'E',
                            'bandgap_energy_ev' : 'Eg'}
    
    
df_train = pd.read_csv("./input/train.csv").rename(columns=columns)
df_train["dataset"] = "train"
df_train["E"]=np.log1p(df_train["E"])
df_train["Eg"]=np.log1p(df_train["Eg"])
df_test = pd.read_csv("./input/test.csv").rename(columns=columns)
df_test["dataset"] = "test"
df_total = pd.concat([df_train, df_test], ignore_index=True)

len(df_train),len(df_test),len(df_total)

(2400, 600, 3000)

In [3]:
df_total.head()

,E,Eg,Natoms,a,alpha,b,beta,c,dataset,gamma,id,sg,x_Al,x_Ga,x_In
0,0.065788,1.490362,80.0,9.9523,90.0026,8.5513,90.0023,9.1775,train,90.0017,1,33,0.6250,0.3750,0.000
1,0.222343,1.366347,80.0,6.1840,90.0186,6.1838,89.9980,23.6287,train,120.0025,2,194,0.6250,0.3750,0.000
2,0.167293,1.320101,40.0,9.7510,90.9688,5.6595,91.1228,13.9630,train,30.5185,3,227,0.8125,0.1875,0.000
3,0.196553,1.469992,30.0,5.0036,89.9888,5.0034,90.0119,13.5318,train,120.0017,4,167,0.7500,0.0000,0.250
4,0.049266,0.866806,80.0,6.6614,89.9960,6.6612,90.0006,24.5813,train,119.9893,5,194,0.0000,0.6250,0.375


In [4]:
df_total.tail()

,E,Eg,Natoms,a,alpha,b,beta,c,dataset,gamma,id,sg,x_Al,x_Ga,x_In
2995,NaN,NaN,80.0,24.8145,90.0002,6.3964,104.7733,6.2933,test,90.0001,596,12,0.000,0.5938,0.4062
2996,NaN,NaN,40.0,5.5783,90.0008,9.4849,89.9967,10.1107,test,90.0004,597,33,0.125,0.0000,0.8750
2997,NaN,NaN,80.0,6.9377,90.0072,6.9372,89.9880,25.0641,test,119.9857,598,194,0.000,0.2500,0.7500
2998,NaN,NaN,40.0,5.1841,90.0041,8.8659,90.0009,9.4956,test,90.0007,599,33,0.625,0.0000,0.3750
2999,NaN,NaN,80.0,9.4959,90.0029,9.4956,90.0031,9.4956,test,89.9969,600,206,0.375,0.3438,0.2812


In [5]:
#from https://www.kaggle.com/cbartel/random-forest-using-elemental-properties
def get_vol(a, b, c, alpha, beta, gamma):
    """
    Args:
        a (float) - lattice vector 1
        b (float) - lattice vector 2
        c (float) - lattice vector 3
        alpha (float) - lattice angle 1 [radians]
        beta (float) - lattice angle 2 [radians]
        gamma (float) - lattice angle 3 [radians]
    Returns:
        volume (float) of the parallelepiped unit cell
    """
    alpha=alpha*np.pi/180
    beta=beta*np.pi/180
    gamma=gamma*np.pi/180
    return a*b*c*np.sqrt(1 + 2*np.cos(alpha)*np.cos(beta)*np.cos(gamma)
                           - np.cos(alpha)**2
                           - np.cos(beta)**2
                           - np.cos(gamma)**2)


    
# compute the cell volumes 
df_total['vol'] = get_vol(df_total['a'], df_total['b'], df_total['c'],
                          df_total['alpha'], df_total['beta'], df_total['gamma'])
#df_total[['a','b','c','alpha','beta','gamma','vol']].head()
df_total['density']=df_total['Natoms']/df_total["vol"]
df_total['density_Al']=df_total['density']*df_total['x_Al']
df_total['density_Ga']=df_total['density']*df_total['x_Ga']
df_total['density_In']=df_total['density']*df_total['x_In']
df_total['sg']=df_total['sg'].astype('category')

In [6]:
df_total.head()

,E,Eg,Natoms,a,alpha,b,beta,c,dataset,gamma,id,sg,x_Al,x_Ga,x_In,vol,density,density_Al,density_Ga,density_In
0,0.065788,1.490362,80.0,9.9523,90.0026,8.5513,90.0023,9.1775,train,90.0017,1,33,0.6250,0.3750,0.000,781.052081,0.102426,0.064016,0.038410,0.000000
1,0.222343,1.366347,80.0,6.1840,90.0186,6.1838,89.9980,23.6287,train,120.0025,2,194,0.6250,0.3750,0.000,782.500110,0.102236,0.063898,0.038339,0.000000
2,0.167293,1.320101,40.0,9.7510,90.9688,5.6595,91.1228,13.9630,train,30.5185,3,227,0.8125,0.1875,0.000,391.227531,0.102242,0.083072,0.019170,0.000000
3,0.196553,1.469992,30.0,5.0036,89.9888,5.0034,90.0119,13.5318,train,120.0017,4,167,0.7500,0.0000,0.250,293.377334,0.102257,0.076693,0.000000,0.025564
4,0.049266,0.866806,80.0,6.6614,89.9960,6.6612,90.0006,24.5813,train,119.9893,5,194,0.0000,0.6250,0.375,944.713843,0.084682,0.000000,0.052926,0.031756


In [7]:
#Encoding of cat features
import sys 
sys.path.append("../kaggle_varie")
from  varie import *
cols_to_enc=["sg"]

#binary encoder
#enc=bin_enc(df_total,cols_to_enc,verbose=2,copy=True,drop_original=True,ordinal_only=False)
#one-hot encoder
enc=pd.get_dummies(df_total,columns=cols_to_enc)

/scratch/baruselli/inst/intelpython35/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/scratch/baruselli/inst/intelpython35/lib/python3.5/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/scratch/baruselli/inst/intelpython35/lib/python3.5/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions a

In [8]:
def grid_search_fct(model,params,df,y_col,n_iter=20,cv=4,drop_col=[],verbose=2):
    
    X_train=df.drop(y_col+drop_col,axis=1).values
    grids=[]
    for y in y_col:
        print(y)
        y_train=df[y].values
        print(X_train.shape,y_train.shape)

        grid=RandomizedSearchCV(model,param_distributions=params, n_iter=n_iter,cv=cv,verbose=verbose,scoring="neg_mean_squared_error" )

        grid.fit(X_train,y_train)
        grids.append(grid)
    return grids

In [40]:
from sklearn.linear_model import  ElasticNet
ElasticNet?

In [347]:
ensemble.RandomForestRegressor?

In [348]:
#grid search for random forest
import scipy
from  sklearn.model_selection import RandomizedSearchCV
from sklearn import *
from catboost import CatBoostRegressor,CatBoostClassifier
from sklearn.svm import SVR
from sklearn.linear_model import  ElasticNet
from sklearn.ensemble import  GradientBoostingRegressor, RandomForestRegressor,AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
#from sklearn.kernel_approximation import Nystroem
#from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import Lasso,Ridge
from xgboost import XGBRegressor
#from lightgbm import LGBMRegressor
from varie import lognuniform
from varie import loguniform2
%aimport varie

y_col=["E","Eg"]
drop_col=["id","dataset"]
df_total_train_eval=enc[df_total.dataset=='train']
df_total_test=enc[df_total.dataset=='test']

X_train=df_total_train_eval.drop(y_col+drop_col,axis=1).values
X_test=df_total_test.drop(y_col+drop_col,axis=1).values

models={
    
    'knn':
           (KNeighborsRegressor(),
            {'n_neighbors':scipy.stats.randint(1,100)}),
    
    'svr':
           (SVR(verbose=False),
            {'C':loguniform(low=-4,high=4,base=10,size=100),
                'epsilon':lognuniform(low=-2,high=0,base=10,size=100)}),

    'rf':
           (ensemble.RandomForestRegressor(verbose=False),
            {"max_depth": scipy.stats.randint(1,100), 
             'n_estimators': scipy.stats.randint(1,400),
             'max_features':('log2','sqrt','auto'),
             'min_samples_split':scipy.stats.randint(2,5),
             'min_samples_leaf':scipy.stats.randint(1,5)}),
    
    'cb':
           (CatBoostRegressor(loss_function='RMSE', eval_metric='RMSE',logging_level='Silent'),
            {"depth": scipy.stats.randint(1,6), 
             'iterations': scipy.stats.randint(100,2000),
             'learning_rate':lognuniform(low=-2,high=-1,base=10,size=100),
             'l2_leaf_reg': scipy.stats.randint(2,4)}),
    
    'mlp': 
           (MLPRegressor((80, 10), early_stopping=False),
             {'hidden_layer_sizes':scipy.stats.randint(1,100),
              'alpha':lognuniform(low=-5,high=-1,base=10,size=100)}),
             
     'gb':
           (GradientBoostingRegressor(n_estimators=100),
            {'learning_rate':lognuniform(low=-3,high=-1,base=10,size=100), 
             'n_estimators': scipy.stats.randint(1,300),
             'max_depth':scipy.stats.randint(1,5),
             'max_features':('sqrt','log2','auto')}),
    
    'lasso':
            (Lasso(),
            {'alpha':lognuniform(low=-6,high=2,base=10,size=100)}),  

    'ridge':
            (Ridge(),
            {'alpha':varie.lognuniform(low=-6,high=2,base=10,size=100)}),
    
    'eln':
            (ElasticNet(),
            {'alpha':lognuniform(low=-6,high=4,base=10,size=100), 
             'l1_ratio':lognuniform(low=-6,high=4,base=10,size=100)}),
    
    'xgb':
        (XGBRegressor(),
         {'max_depth':scipy.stats.randint(1,100), 
          'learning_rate':lognuniform(low=-4,high=-0.5,base=10,size=100), 
          'n_estimators':scipy.stats.randint(1,400),
          'colsample_bytree': uniform(0.55, 0.66),
          'min_child_weight': randint(30, 60),
          'colsample_bytree': uniform(0.6, 0.4),
          'reg_lambda': uniform(1, 2),
          'reg_alpha': uniform(1, 2),
}),
    
 #does not install    
 #   'gbm' :
 #       (lgb.LGBMRegressor(objective='regression'),
 #           {'num_leaves':scipy.stats.randint(1,200), 
 #         'learning_rate':lognuniform(low=-4,high=-0.5,base=10,size=100), 
 #         'n_estimators':scipy.stats.randint(1,400)}),
    'adb' :
        (AdaBoostRegressor(loss="square"),
            {'learning_rate':lognuniform(low=-4,high=-0.1,base=10,size=100), 
             'n_estimators':scipy.stats.randint(1,400)}),         

    
       }
 
    

try:
    results
except:
    results={}
    
for (tag,model) in  models.items():
    if (tag not in results):
        print(tag)
        results[tag]=grid_search_fct(model[0],model[1],df_total_train_eval,y_col,n_iter=100,cv=4,drop_col=drop_col)


    
    #grid=RandomizedSearchCV(model[0],param_distributions=params, n_iter=20,cv=4,verbose=2,scoring="neg_mean_squared_error" )

                        
    #grid.fit(X_train,y_train)
    #grids.append(grid)

cb
E
(2400, 21) (2400,)
Fitting 4 folds for each of 100 candidates, totalling 400 fits
[CV] depth=4, iterations=1187, learning_rate=0.01583042064951216, l2_leaf_reg=3 
[CV]  depth=4, iterations=1187, learning_rate=0.01583042064951216, l2_leaf_reg=3, total=   5.0s
[CV] depth=4, iterations=1187, learning_rate=0.01583042064951216, l2_leaf_reg=3 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.0s remaining:    0.0s


[CV]  depth=4, iterations=1187, learning_rate=0.01583042064951216, l2_leaf_reg=3, total=   4.2s
[CV] depth=4, iterations=1187, learning_rate=0.01583042064951216, l2_leaf_reg=3 
[CV]  depth=4, iterations=1187, learning_rate=0.01583042064951216, l2_leaf_reg=3, total=   4.4s
[CV] depth=4, iterations=1187, learning_rate=0.01583042064951216, l2_leaf_reg=3 
[CV]  depth=4, iterations=1187, learning_rate=0.01583042064951216, l2_leaf_reg=3, total=   4.4s
[CV] depth=3, iterations=1765, learning_rate=0.028513910607754597, l2_leaf_reg=2 
[CV]  depth=3, iterations=1765, learning_rate=0.028513910607754597, l2_leaf_reg=2, total=   4.9s
[CV] depth=3, iterations=1765, learning_rate=0.028513910607754597, l2_leaf_reg=2 
[CV]  depth=3, iterations=1765, learning_rate=0.028513910607754597, l2_leaf_reg=2, total=   6.2s
[CV] depth=3, iterations=1765, learning_rate=0.028513910607754597, l2_leaf_reg=2 
[CV]  depth=3, iterations=1765, learning_rate=0.028513910607754597, l2_leaf_reg=2, total=   5.0s
[CV] depth=3,

[CV]  depth=2, iterations=1607, learning_rate=0.09987135664906072, l2_leaf_reg=3, total=   3.9s
[CV] depth=2, iterations=1607, learning_rate=0.09987135664906072, l2_leaf_reg=3 
[CV]  depth=2, iterations=1607, learning_rate=0.09987135664906072, l2_leaf_reg=3, total=   3.8s
[CV] depth=2, iterations=1607, learning_rate=0.09987135664906072, l2_leaf_reg=3 
[CV]  depth=2, iterations=1607, learning_rate=0.09987135664906072, l2_leaf_reg=3, total=   3.9s
[CV] depth=2, iterations=1607, learning_rate=0.09987135664906072, l2_leaf_reg=3 
[CV]  depth=2, iterations=1607, learning_rate=0.09987135664906072, l2_leaf_reg=3, total=   3.8s
[CV] depth=3, iterations=1398, learning_rate=0.010445739850637821, l2_leaf_reg=3 
[CV]  depth=3, iterations=1398, learning_rate=0.010445739850637821, l2_leaf_reg=3, total=   4.0s
[CV] depth=3, iterations=1398, learning_rate=0.010445739850637821, l2_leaf_reg=3 
[CV]  depth=3, iterations=1398, learning_rate=0.010445739850637821, l2_leaf_reg=3, total=   4.0s
[CV] depth=3, i

[CV]  depth=4, iterations=1812, learning_rate=0.016450140352136634, l2_leaf_reg=3, total=   6.2s
[CV] depth=4, iterations=490, learning_rate=0.07242175416325623, l2_leaf_reg=3 
[CV]  depth=4, iterations=490, learning_rate=0.07242175416325623, l2_leaf_reg=3, total=   2.2s
[CV] depth=4, iterations=490, learning_rate=0.07242175416325623, l2_leaf_reg=3 
[CV]  depth=4, iterations=490, learning_rate=0.07242175416325623, l2_leaf_reg=3, total=   2.2s
[CV] depth=4, iterations=490, learning_rate=0.07242175416325623, l2_leaf_reg=3 
[CV]  depth=4, iterations=490, learning_rate=0.07242175416325623, l2_leaf_reg=3, total=   2.1s
[CV] depth=4, iterations=490, learning_rate=0.07242175416325623, l2_leaf_reg=3 
[CV]  depth=4, iterations=490, learning_rate=0.07242175416325623, l2_leaf_reg=3, total=   2.1s
[CV] depth=3, iterations=1405, learning_rate=0.04750111202621063, l2_leaf_reg=2 
[CV]  depth=3, iterations=1405, learning_rate=0.04750111202621063, l2_leaf_reg=2, total=   4.0s
[CV] depth=3, iterations=1

...

[CV]  depth=1, iterations=925, learning_rate=0.029683227760220216, l2_leaf_reg=3, total=   2.0s
[CV] depth=1, iterations=925, learning_rate=0.029683227760220216, l2_leaf_reg=3 
[CV]  depth=1, iterations=925, learning_rate=0.029683227760220216, l2_leaf_reg=3, total=   2.1s
[CV] depth=1, iterations=925, learning_rate=0.029683227760220216, l2_leaf_reg=3 
[CV]  depth=1, iterations=925, learning_rate=0.029683227760220216, l2_leaf_reg=3, total=   2.1s
[CV] depth=1, iterations=925, learning_rate=0.029683227760220216, l2_leaf_reg=3 
[CV]  depth=1, iterations=925, learning_rate=0.029683227760220216, l2_leaf_reg=3, total=   2.2s
[CV] depth=3, iterations=1003, learning_rate=0.031775560054999395, l2_leaf_reg=2 
[CV]  depth=3, iterations=1003, learning_rate=0.031775560054999395, l2_leaf_reg=2, total=   3.8s
[CV] depth=3, iterations=1003, learning_rate=0.031775560054999395, l2_leaf_reg=2 
[CV]  depth=3, iterations=1003, learning_rate=0.031775560054999395, l2_leaf_reg=2, total=   3.2s
[CV] depth=3, i

[CV]  depth=4, iterations=781, learning_rate=0.08625258753800176, l2_leaf_reg=3, total=   3.2s
[CV] depth=1, iterations=236, learning_rate=0.022761243586365002, l2_leaf_reg=2 
[CV]  depth=1, iterations=236, learning_rate=0.022761243586365002, l2_leaf_reg=2, total=   1.0s
[CV] depth=1, iterations=236, learning_rate=0.022761243586365002, l2_leaf_reg=2 
[CV]  depth=1, iterations=236, learning_rate=0.022761243586365002, l2_leaf_reg=2, total=   1.0s
[CV] depth=1, iterations=236, learning_rate=0.022761243586365002, l2_leaf_reg=2 
[CV]  depth=1, iterations=236, learning_rate=0.022761243586365002, l2_leaf_reg=2, total=   0.9s
[CV] depth=1, iterations=236, learning_rate=0.022761243586365002, l2_leaf_reg=2 
[CV]  depth=1, iterations=236, learning_rate=0.022761243586365002, l2_leaf_reg=2, total=   0.9s
[CV] depth=4, iterations=1348, learning_rate=0.0932220847391707, l2_leaf_reg=2 
[CV]  depth=4, iterations=1348, learning_rate=0.0932220847391707, l2_leaf_reg=2, total=   4.8s
[CV] depth=4, iteratio

[CV]  depth=4, iterations=1221, learning_rate=0.017424851332484684, l2_leaf_reg=2, total=   4.6s
[CV] depth=4, iterations=1221, learning_rate=0.017424851332484684, l2_leaf_reg=2 
[CV]  depth=4, iterations=1221, learning_rate=0.017424851332484684, l2_leaf_reg=2, total=   4.5s
[CV] depth=3, iterations=1040, learning_rate=0.04684581468329573, l2_leaf_reg=3 
[CV]  depth=3, iterations=1040, learning_rate=0.04684581468329573, l2_leaf_reg=3, total=   3.2s
[CV] depth=3, iterations=1040, learning_rate=0.04684581468329573, l2_leaf_reg=3 
[CV]  depth=3, iterations=1040, learning_rate=0.04684581468329573, l2_leaf_reg=3, total=   3.2s
[CV] depth=3, iterations=1040, learning_rate=0.04684581468329573, l2_leaf_reg=3 
[CV]  depth=3, iterations=1040, learning_rate=0.04684581468329573, l2_leaf_reg=3, total=   3.1s
[CV] depth=3, iterations=1040, learning_rate=0.04684581468329573, l2_leaf_reg=3 
[CV]  depth=3, iterations=1040, learning_rate=0.04684581468329573, l2_leaf_reg=3, total=   3.2s
[CV] depth=5, it

[CV]  depth=3, iterations=1920, learning_rate=0.03465140637006451, l2_leaf_reg=3, total=   5.3s
[CV] depth=3, iterations=1920, learning_rate=0.03465140637006451, l2_leaf_reg=3 
[CV]  depth=3, iterations=1920, learning_rate=0.03465140637006451, l2_leaf_reg=3, total=   5.5s
[CV] depth=3, iterations=1920, learning_rate=0.03465140637006451, l2_leaf_reg=3 
[CV]  depth=3, iterations=1920, learning_rate=0.03465140637006451, l2_leaf_reg=3, total=   5.2s
[CV] depth=5, iterations=366, learning_rate=0.017424851332484684, l2_leaf_reg=3 
[CV]  depth=5, iterations=366, learning_rate=0.017424851332484684, l2_leaf_reg=3, total=   2.0s
[CV] depth=5, iterations=366, learning_rate=0.017424851332484684, l2_leaf_reg=3 
[CV]  depth=5, iterations=366, learning_rate=0.017424851332484684, l2_leaf_reg=3, total=   2.0s
[CV] depth=5, iterations=366, learning_rate=0.017424851332484684, l2_leaf_reg=3 
[CV]  depth=5, iterations=366, learning_rate=0.017424851332484684, l2_leaf_reg=3, total=   2.1s
[CV] depth=5, itera

[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed: 23.0min finished


In [344]:
import pickle
#pickle.dump(results, open( "results_100iter.pickle", "wb" ))
#results.pop('cb')

[RandomizedSearchCV(cv=4, error_score='raise',
           estimator=<catboost.core.CatBoostRegressor object at 0x7ff018b4c3c8>,
           fit_params=None, iid=True, n_iter=100, n_jobs=1,
           param_distributions={'depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff018b4c4a8>, 'iterations': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff018b4c588>, 'learning_rate': array([0.06793, 0.02726, ..., 0.01334, 0.01936])},
           pre_dispatch='2*n_jobs', random_state=None, refit=True,
           return_train_score='warn', scoring='neg_mean_squared_error',
           verbose=2), RandomizedSearchCV(cv=4, error_score='raise',
           estimator=<catboost.core.CatBoostRegressor object at 0x7ff018b4c3c8>,
           fit_params=None, iid=True, n_iter=100, n_jobs=1,
           param_distributions={'depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff018b4c4a8>, 'iterations': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff018b4c588

In [349]:
#best models and their performance

for tag,grids in results.items():
    print(tag)
    for grid in grids:
        print(grid.best_params_)
    print((np.sqrt(-grids[0].best_score_)+np.sqrt(-grids[1].best_score_))/2)

xgb
{'min_child_weight': 50, 'max_depth': 20, 'reg_alpha': 1.073947179242888, 'colsample_bytree': 0.9495387910705909, 'n_estimators': 82, 'learning_rate': 0.19564220615889624, 'reg_lambda': 2.5750775958117913}
{'min_child_weight': 59, 'max_depth': 47, 'reg_alpha': 1.0745756197034326, 'colsample_bytree': 0.894792479949726, 'n_estimators': 394, 'learning_rate': 0.021394988524502393, 'reg_lambda': 1.4038745160788761}
0.06070531708533062
svr
{'C': 7.714813468681621, 'epsilon': 0.019583331445081988}
{'C': 0.3418951606057705, 'epsilon': 0.02841600304754715}
0.094502440393147
mlp
{'alpha': 0.013697209704446447, 'hidden_layer_sizes': 98}
{'alpha': 1.2764744090326039e-05, 'hidden_layer_sizes': 98}
0.1470021188220108
rf
{'max_features': 'log2', 'max_depth': 9, 'n_estimators': 267}
{'max_features': 'sqrt', 'max_depth': 8, 'n_estimators': 336}
0.06256737960363293
ridge
{'alpha': 0.0003364106604226533}
{'alpha': 0.0005566509619678715}
0.06670031387253723
knn
{'n_neighbors': 5}
{'n_neighbors': 3}
0.

(XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
        learning_rate=0.1, max_delta_step=0, max_depth=3,
        min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
        objective='reg:linear', reg_alpha=0, reg_lambda=1,
        scale_pos_weight=1, seed=0, silent=True, subsample=1),
 {'colsample_bytree': <scipy.stats._distn_infrastructure.rv_frozen at 0x7fefee4885c0>,
  'learning_rate': array([8.51449911e-03, 5.11998252e-02, 1.65751975e-01, 5.45008940e-02,
         2.36368993e-01, 6.05695533e-04, 2.66160637e-03, 2.85826030e-03,
         6.39575488e-04, 2.49487140e-03, 3.08001078e-03, 1.34662552e-03,
         4.22702713e-02, 1.70922580e-02, 2.23401003e-01, 1.07362617e-02,
         1.20987221e-04, 1.33759572e-03, 3.73324526e-02, 8.24032738e-04,
         2.30313264e-02, 1.30357543e-01, 1.13102410e-03, 1.81017102e-03,
         1.68583754e-04, 4.32220730e-03, 1.32185029e-04, 2.51015261e-01,
         1.52757643e-01, 1.07889541e-01, 5.91861241e-

In [373]:
learners1

[XGBRegressor(base_score=0.5, colsample_bylevel=1,
        colsample_bytree=0.9495387910705909, gamma=0,
        learning_rate=0.19564220615889624, max_delta_step=0, max_depth=20,
        min_child_weight=50, missing=None, n_estimators=82, nthread=-1,
        objective='reg:linear', reg_alpha=1.073947179242888,
        reg_lambda=2.5750775958117913, scale_pos_weight=1, seed=0,
        silent=True, subsample=1),
 SVR(C=7.714813468681621, cache_size=200, coef0=0.0, degree=3,
   epsilon=0.019583331445081988, gamma='auto', kernel='rbf', max_iter=-1,
   shrinking=True, tol=0.001, verbose=False),
 MLPRegressor(activation='relu', alpha=0.013697209704446447, batch_size='auto',
        beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
        hidden_layer_sizes=98, learning_rate='constant',
        learning_rate_init=0.001, max_iter=200, momentum=0.9,
        nesterovs_momentum=True, power_t=0.5, random_state=None,
        shuffle=True, solver='adam', tol=0.0001, validation_fractio

In [396]:
meta_learner=CatBoostRegressor(loss_function='RMSE', eval_metric='RMSE',logging_level='Silent')

params_meta={'meta-me__depth': scipy.stats.randint(1,6)}
             
             
             #'meta-meta_learner__iterations': scipy.stats.randint(100,2000),
             #'meta-meta_learner__learning_rate':lognuniform(low=-2,high=-1,base=10,size=100),
             #'meta-meta_learner__l2_leaf_reg': scipy.stats.randint(2,4)}
            
from mlxtend.regressor import StackingRegressor

learners1=[g[0].best_estimator_ for g in results.values()]
learners2=[g[1].best_estimator_ for g in results.values()]
learners=[learners1,learners2]

stregr = [StackingRegressor(regressors=learners1,meta_regressor=meta_learner),
          StackingRegressor(regressors=learners2,meta_regressor=meta_learner)]




results2={}
for s,y in zip(stregr,y_col):
    #print(s,y)
    y_train=df_total_train_eval[y]
    grid=RandomizedSearchCV(s,param_distributions=params_meta, n_iter=10,cv=5,verbose=1,scoring="neg_mean_squared_error" )
    grid.fit(X_train, y_train)
    results2.append(grid)



Fitting 5 folds for each of 10 candidates, totalling 50 fits


ValueError: Invalid parameter meta-Cat for estimator StackingRegressor(meta_regressor=<catboost.core.CatBoostRegressor object at 0x7fefee38a470>,
         regressors=[XGBRegressor(base_score=0.5, colsample_bylevel=1,
       colsample_bytree=0.9495387910705909, gamma=0,
       learning_rate=0.19564220615889624, max_delta_step=0, max_depth=20,
       min_child_weight=50, missing=None, n_estimators=82, nthread=-1,
       objective='reg:linear', reg_alpha...ne, learning_rate=0.13761901851857872,
         loss='square', n_estimators=211, random_state=None)],
         store_train_meta_features=False, verbose=0). Check the list of available parameters with `estimator.get_params().keys()`.

In [329]:
from mlens.ensemble import SuperLearner
import mlens
from mlens.model_selection import Evaluator
from mlens.metrics import make_scorer
from mlens.metrics import rmse

from mlens.metrics import make_scorer
rmse_scorer = make_scorer(rmse, greater_is_better=False)



learners1=[g[0].best_estimator_ for g in results.values()]
learners2=[g[1].best_estimator_ for g in results.values()]
learners=[learners1,learners2]

# Instantiate the ensemble with 10 folds
#meta_learner1=CatBoostRegressor(iterations=1200,
#                            learning_rate=0.03,
#                            depth=4,
#                            loss_function='RMSE',
#                            eval_metric='RMSE',
##                            random_seed=SEED,
#                            od_type='Iter',
#                            od_wait=50,verbose=False)

#import copy
#meta_learner2=copy.deepcopy(meta_learner1)

#sl1 = SuperLearner(
#    folds=5,
#    verbose=True,
##    scorer=mlens.metrics.rmse
#)
#sl2 = SuperLearner(
#    folds=5,
#    verbose=True,
#    scorer=mlens.metrics.rmse
#)

# Add the base learners and the meta learner
#sl1.add(learners1) 
#sl1.add_meta(meta_learner1)
#sl2.add(learners2) 
#sl2.add_meta(meta_learner2)

#sls=[sl1,sl2]
#evaluator
#evl = Evaluator(make_scorer(mlens.metrics.rmse), cv=5, shuffle=False)
sls=[]
for learner,y in zip(learners,y_col):
    print(y)
    y_train=df_total_train_eval[y].values
    print(X_train.shape,y_train.shape)
    
    #evl.fit(X_train, y_train, sl, {}, n_iter=1)
    from mlens.ensemble import SuperLearner
    
    sl = SuperLearner(
    folds=5,
    verbose=True,
    #    scorer=mlens.metrics.rmse
    )
    
    meta_learner=CatBoostRegressor(iterations=1200,
                            learning_rate=0.03,
                            depth=4,
                            loss_function='RMSE',
                            eval_metric='RMSE',
#                            random_seed=SEED,
                            od_type='Iter',
                            od_wait=50,verbose=False)
    
    sl.add(learner) 
    sl.add_meta(meta_learner)
    # Train the ensemble
    sl.fit(X_train, y_train)
    preds = sl.predict(X_train)
    print(rmse(y_train, preds))
    sls.append(sl)
#    results.append(mlens.metrics.rmse(y_train, ensemble.predict(X_train)),
#                          evl.summary['test_score_mean']['superlearner'],
#                          evl.summary['test_score_std']['superlearner'],
#                          mlens.metrics.rmse(y_test, ensemble.predict(X_test)))

#    print_scores(scores_df, 'mlens')

E
(2400, 21) (2400,)

Fitting 2 layers


JoblibIndexError: JoblibIndexError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/scratch/baruselli/inst/intelpython35/lib/python3.5/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    179         sys.exit(msg)
    180     main_globals = sys.modules["__main__"].__dict__
    181     if alter_argv:
    182         sys.argv[0] = mod_spec.origin
    183     return _run_code(code, main_globals, None,
--> 184                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.5/site-packages/ipykernel_launcher.py')
    185 
    186 def run_module(mod_name, init_globals=None,
    187                run_name=None, alter_sys=False):
    188     """Execute a module's code without importing it

...........................................................................
/scratch/barus

In [ ]:
## from mlens.ensemble import SuperLearner
import mlens
from mlens.model_selection import Evaluator
from mlens.metrics import make_scorer
from mlens.metrics import rmse



learners1=[grid[0].best_estimator_ for grid in results.values()]
learners2=[grid[1].best_estimator_ for grid in results.values()]

grid_sl=[]

tries=3
#grid search for the meta learner         
for depth, iterations, learning_rate in zip (scipy.stats.randint(1,5).rvs(tries),
                                              scipy.stats.randint(1000,2000).rvs(tries),    
                                              lognuniform(low=-2,high=-1,base=10,size=tries)):
    print(depth, iterations, learning_rate)
    

    # Instantiate the ensemble with 10 folds
    meta_learner1=CatBoostRegressor(iterations=iterations,
                                learning_rate=learning_rate,
                                depth=depth,
                                loss_function='RMSE',
                                eval_metric='RMSE',
    #                            random_seed=SEED,
                                od_type='Iter',
                                od_wait=50,verbose=False)

    import copy
    meta_learner2=copy.deepcopy(meta_learner1)

    sl1 = SuperLearner(
        folds=5,
        verbose=True,
        scorer=mlens.metrics.rmse
    )
    sl2 = SuperLearner(
        folds=5,
        verbose=True,
        scorer=mlens.metrics.rmse
    )

    # Add the base learners and the meta learner
    sl1.add(learners1) 
    sl1.add_meta(meta_learner1)
    sl2.add(learners2) 
    sl2.add_meta(meta_learner2)



    sls=[sl1,sl2]
    #evaluator
    #evl = Evaluator(make_scorer(mlens.metrics.rmse), cv=5, shuffle=False)

    for i,y in enumerate(y_col):
        print(y)
        y_train=df_total_train_eval[y].values
        #print(X_train.shape,y_train.shape)

        #evl.fit(X_train, y_train, sl, {}, n_iter=1)

        # Train the ensemble
        sls[i].fit(X_train, y_train)
        preds = sls[i].predict(X_train)
        print(rmse(y_train, preds))
        
        grid_sl.append(depth, iterations, learning_rate,sls)
        
        
    #    results.append(mlens.metrics.rmse(y_train, ensemble.predict(X_train)),
    #                          evl.summary['test_score_mean']['superlearner'],
    #                          evl.summary['test_score_std']['superlearner'],
    #                          mlens.metrics.rmse(y_test, ensemble.predict(X_test)))

    #    print_scores(scores_df, 'mlens')

In [272]:
in_layer = SuperLearner(model_selection=True)
in_layer.add(base_learners)

preprocess = [in_layer]

evl = Evaluator(
    scorer,
    cv=2,
    verbose=5,
)

evl.fit(
    X_train, y_train,
    meta_learners,
    param_dicts,
    preprocessing={'meta': preprocess},
    n_iter=5                           # bump this up to do a larger grid search
)


Launching job
Preprocessing 1 preprocessing pipelines over 2 CV folds


JoblibIndexError: JoblibIndexError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/scratch/baruselli/inst/intelpython35/lib/python3.5/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    179         sys.exit(msg)
    180     main_globals = sys.modules["__main__"].__dict__
    181     if alter_argv:
    182         sys.argv[0] = mod_spec.origin
    183     return _run_code(code, main_globals, None,
--> 184                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.5/site-packages/ipykernel_launcher.py')
    185 
    186 def run_module(mod_name, init_globals=None,
    187                run_name=None, alter_sys=False):
    188     """Execute a module's code without importing it

...........................................................................
/scratch/barus

In [136]:
#write to csv
%load_ext autoreload
%aimport varie
%autoreload 2
#I use a different model for E and Eg
varie.make_csv2(df_total_train_eval,pd.DataFrame(),df_total_test,
#         (ensemble.RandomForestRegressor(max_depth= 11, max_features='log2', n_estimators= 55),
#          ensemble.RandomForestRegressor(max_depth= 9, max_features='sqrt', n_estimators= 220)),
            (sl1,sl2),
         y_col,'sl3.csv',drop=drop_col,columns=['id','E','Eg'],
         new_column_names=['id','formation_energy_ev_natom' ,'bandgap_energy_ev'],change_col_names=True,fit=False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
E SuperLearner(array_check=2, backend=None, folds=5,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=None, shuffle=False,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=FoldIndex(X=None, folds=5, raise_on_ex...d37840>)],
   n_jobs=-1, name='group-21', raise_on_exception=True, transformers=[])],
   verbose=0)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=None, sample_size=20,
       scorer=<function rmse at 0x7fefeed37840>, shuffle=False,
       verbose=True)
shapes: (2400, 21) (2400,)

Predicting 2 layers
Predict complete                    | 00:00:00
Eg SuperLearner(array_check=2, backend=None, folds=5,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', 

In [ ]:
grids_sl=[]
for i,y in enumerate(y_col):
    print(y)
    y_train=df_total_train_eval[y].values
    print(X_train.shape,y_train.shape)

    grid=RandomizedSearchCV(model,param_distributions=params, n_iter=n_iter,cv=cv,verbose=verbose,scoring="neg_mean_squared_error" )

    grid.fit(X_train,y_train)
    grids_sl.append(grid)

    sls[i].fit(X_train, y_train)
    preds = sls[i].predict(X_train)
    print(rmse(y_train, preds))
#    results.append(mlens.metrics.rmse(y_train, ensemble.predict(X_train)),
#                          evl.summary['test_score_mean']['superlearner'],
#                          evl.summary['test_score_std']['superlearner'],
#                          mlens.metrics.rmse(y_test, ensemble.predict(X_test)))

#    print_scores(scores_df, 'mlens')   X_train=df.drop(y_col+drop_col,axis=1).values
    for y in y_col:

    return grids

In [ ]:
evaluator = Evaluator()
evaluator.fit(X_train, y_train,sl)

In [188]:
models

{'adb': (AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='square',
           n_estimators=50, random_state=None),
  {'learning_rate': <varie.loguniform at 0x7fefeceb89e8>,
   'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen at 0x7fefeceb8710>}),
 'cb': (<catboost.core.CatBoostRegressor at 0x7fefece96828>,
  {'depth': <scipy.stats._distn_infrastructure.rv_frozen at 0x7fefece96908>,
   'iterations': <scipy.stats._distn_infrastructure.rv_frozen at 0x7fefece969e8>,
   'learning_rate': <varie.loguniform at 0x7fefece96ba8>}),
 'eln': (ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
        max_iter=1000, normalize=False, positive=False, precompute=False,
        random_state=None, selection='cyclic', tol=0.0001, warm_start=False),
  {'alpha': <varie.loguniform at 0x7fefeceb81d0>,
   'l1_ratio': <varie.loguniform at 0x7fefeceb8278>}),
 'gb': (GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
               learning_rate=

In [190]:
ests=[(tag, model[0]) for tag,model in models.items()]
params={tag:model[1] for tag,model in models.items() }

In [192]:
from mlens.model_selection import Evaluator

from scipy.stats import randint

# Here we name the estimators ourselves
#ests = [('gnb', GaussianNB()), ('knn', KNeighborsClassifier())]

# Now we map parameters to these
# The gnb doesn't have any parameters so we can skip it
#pars = {'n_neighbors': randint(2, 20)}
#params = {'knn': pars}

evaluators=[]
for i,y in enumerate(y_col):
    print(y)
    y_train=df_total_train_eval[y].values
    print(X_train.shape,y_train.shape)
    evaluator = Evaluator(rmse_scorer, cv=10,  verbose=1)

    
    evaluator.fit(X_train,y_train, ests, params, n_iter=5)
    evaluators.append(evaluator)

E
(2400, 21) (2400,)


[autoreload of varie failed: Traceback (most recent call last):
  File "/scratch/baruselli/inst/intelpython35/lib/python3.5/site-packages/IPython/extensions/autoreload.py", line 246, in check
    superreload(m, reload, self.old_objects)
  File "/scratch/baruselli/inst/intelpython35/lib/python3.5/site-packages/IPython/extensions/autoreload.py", line 369, in superreload
    module = reload(module)
  File "/scratch/baruselli/inst/intelpython35/lib/python3.5/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/scratch/baruselli/inst/intelpython35/lib/python3.5/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 626, in _exec
  File "<frozen importlib._bootstrap_external>", line 665, in exec_module
  File "<frozen importlib._bootstrap>", line 222, in _call_with_frames_removed
  File "../kaggle_varie/varie.py", line 186, in <module>
    class loguniform:
  File "../kaggle_varie/varie.py", line 196, i

TypeError: rvs() got an unexpected keyword argument 'size'

In [243]:
from mlens.metrics import make_scorer
rmse_scorer = make_scorer(rmse, average='micro', greater_is_better=False)

In [264]:
base_learners=[(tag,model[0]) for tag,model in models.items()]
param_dicts_base={tag:model[1] for tag,model in models.items()}
len(param_dicts_base),len(base_learners)

(11, 11)

In [248]:
from scipy.stats import uniform, randint
SEED=1
# We consider the following models (or base learners)
gb = XGBRegressor()
ls = Lasso(alpha=1e-6, normalize=True)
el = ElasticNet(alpha=1e-6, normalize=True)
rf = RandomForestRegressor(random_state=SEED)

base_learners = [
    ('ls', ls), ('el', el), ('rf', rf), ('gb', gb)
]

# Put their parameter dictionaries in a dictionary with the
# estimator names as keys
param_dicts_base = {
    'ls':
    {'alpha': uniform(1e-6, 1e-5)},
    'el':
    {'alpha': uniform(1e-6, 1e-5),
     'l1_ratio': uniform(0, 1)
    },
    'gb':
    {'learning_rate': uniform(0.02, 0.04),
     'colsample_bytree': uniform(0.55, 0.66),
     'min_child_weight': randint(30, 60),
     'max_depth': randint(3, 7),
     'subsample': uniform(0.4, 0.2),
     'n_estimators': randint(150, 200),
     'colsample_bytree': uniform(0.6, 0.4),
     'reg_lambda': uniform(1, 2),
     'reg_alpha': uniform(1, 2),
    },
    'rf':
    {'max_depth': randint(2, 5),
     'min_samples_split': randint(5, 20),
     'min_samples_leaf': randint(10, 20),
     'n_estimators': randint(50, 100),
     'max_features': uniform(0.6, 0.3)
    }
}


In [252]:
from sklearn.preprocessing import StandardScaler
from mlens.model_selection import Evaluator
assert(len(base_learners)==len(param_dicts_base))
from sklearn.metrics import mean_absolute_error
scorer = make_scorer(mean_absolute_error, greater_is_better=False)

evl = Evaluator(
    scorer,
    cv=2,
    random_state=SEED,
    verbose=5,
)



evl.fit(
    X_train, y_train,
    estimators=base_learners,
    param_dicts=param_dicts_base,
    preprocessing={'sc': [StandardScaler()], 'none': []},
    n_iter=2  # bump this up to do a larger grid search
)




Launching job
Preprocessing 2 preprocessing pipelines over 2 CV folds
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s finished
Preprocessing done | 00:00:00
Evaluating 8 models for 2 parameter draws over 2 preprocessing pipelines and 2 CV folds, totalling 32 fits


JoblibIndexError: JoblibIndexError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/scratch/baruselli/inst/intelpython35/lib/python3.5/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    179         sys.exit(msg)
    180     main_globals = sys.modules["__main__"].__dict__
    181     if alter_argv:
    182         sys.argv[0] = mod_spec.origin
    183     return _run_code(code, main_globals, None,
--> 184                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.5/site-packages/ipykernel_launcher.py')
    185 
    186 def run_module(mod_name, init_globals=None,
    187                run_name=None, alter_sys=False):
    188     """Execute a module's code without importing it

...........................................................................
/scratch/barus

In [157]:
pd.DataFrame(evl.results)

,test_score-m,test_score-s,train_score-m,train_score-s,pred_time-m,pred_time-s,fit_time-m,fit_time-s,params
none.el,-0.090491,0.003486,-0.089310,0.001786,0.000214,0.000025,0.036398,0.013003,"{'alpha': 2.9216451067673484e-06, 'l1_ratio': ..."
none.gb,-0.091834,0.004192,-0.082747,0.001947,0.015173,0.009527,0.553889,0.053751,"{'min_child_weight': 46, 'learning_rate': 0.04..."
none.ls,-0.090211,0.003528,-0.088918,0.001773,0.013295,0.016091,0.087065,0.015598,{'alpha': 2.655238783620573e-06}
none.rf,-0.101206,0.004496,-0.093237,0.001944,0.038409,0.020679,0.375930,0.029805,"{'max_depth': 4, 'max_features': 0.62890412037..."
sc.el,-0.090415,0.003550,-0.089112,0.001791,0.001874,0.002038,0.029383,0.010490,"{'alpha': 3.927727748163378e-06, 'l1_ratio': 0..."
sc.gb,-0.093977,0.004120,-0.086887,0.001693,0.008305,0.004214,0.474971,0.056973,"{'min_child_weight': 32, 'learning_rate': 0.04..."
sc.ls,-0.090822,0.003543,-0.089476,0.001828,0.000239,0.000096,0.016792,0.003647,{'alpha': 8.075902639540653e-06}
sc.rf,-0.115580,0.007073,-0.109046,0.001947,0.005519,0.000407,0.202402,0.026227,"{'max_depth': 3, 'max_features': 0.73818934364..."


In [266]:
meta_learners = [
    ('gb', gb), ('el', el)
]

param_dicts = {
    'el':
    {'alpha': uniform(1e-5, 1),
     'l1_ratio': uniform(0, 1)
    },
    'gb':
    {'learning_rate': uniform(0.01, 0.2),
     'subsample': uniform(0.5, 0.5),
     'reg_lambda': uniform(0.1, 1),
     'n_estimators': randint(10, 100)
    },
}


# Put the layers you don't want to tune into an ensemble with model selection turned on
# Just remember to turn it off when you're done!





In [269]:
len(meta_learners), len(param_dicts)

(2, 2)

In [271]:
in_layer = SuperLearner(model_selection=True)
in_layer.add(base_learners)

preprocess = [in_layer]

evl.fit(
    X_train, y_train,
    meta_learners,
    param_dicts,
#    preprocessing={'meta': preprocess},
    n_iter=5                           # bump this up to do a larger grid search
)



TypeError: fit() takes from 2 to 3 positional arguments but 5 were given

In [257]:
pd.DataFrame(evl.results)

,test_score-m,test_score-s,train_score-m,train_score-s,pred_time-m,pred_time-s,fit_time-m,fit_time-s,params
meta.el,-0.023993,0.000318,-0.024774,0.000724,0.002566,0.001773,1.328225,0.007859,"{'alpha': 0.00012437481734488664, 'l1_ratio': ..."
meta.gb,-0.023334,0.000774,-0.021284,0.000475,0.248742,0.163684,1.312409,0.001766,"{'reg_lambda': 0.517022004702574, 'learning_ra..."


In [124]:
def lognuniform(low=0, high=1, size=None, base=np.exp(1)):
    return np.power(base, np.random.uniform(low, high, size))

In [217]:
uniform.rvs?

In [201]:
np.random.uniform?

In [216]:
scipy.stats.uniform?

[autoreload of varie failed: Traceback (most recent call last):
  File "/scratch/baruselli/inst/intelpython35/lib/python3.5/site-packages/IPython/extensions/autoreload.py", line 246, in check
    superreload(m, reload, self.old_objects)
  File "/scratch/baruselli/inst/intelpython35/lib/python3.5/site-packages/IPython/extensions/autoreload.py", line 369, in superreload
    module = reload(module)
  File "/scratch/baruselli/inst/intelpython35/lib/python3.5/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/scratch/baruselli/inst/intelpython35/lib/python3.5/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 626, in _exec
  File "<frozen importlib._bootstrap_external>", line 661, in exec_module
  File "<frozen importlib._bootstrap_external>", line 767, in get_code
  File "<frozen importlib._bootstrap_external>", line 727, in source_to_code
  File "<frozen importlib._bootstrap>", line 222, in _ca

In [ ]:
from sympy.stats import *
x = Symbol('x')
X = ContinuousRV(x, 2*x, Interval(0, 1))

P(X>.5) 

Var(X) # variance

E(2*cos(X)+X**2) # complex expressions are ok too
